In [2]:
import pandas as pd
import glob
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Step 1: 讀取所有檔案並將它們載入至字典中，每個地點一個DataFrame
file_paths = glob.glob("/mnt/disk2/kuan/new_L*_Train.csv")  # 請替換為實際路徑
location_data = {f"L{idx + 1}": pd.read_csv(file) for idx, file in enumerate(file_paths)}

# 合併所有地點的數據
all_data = pd.concat(location_data.values(), ignore_index=True)


In [3]:
all_data

,DateTime,LocationCode,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),Hour,Minute,Month,Hour_sin,Hour_cos,Minute_sin,Minute_cos,Month_sin,Month_cos,DayOfYear,DayOfYear_sin,DayOfYear_cos
0,2024-04-21 16:32:00,5.0,1.74,1004.50,34.08,51.42,40567.50,332.46,16,32,4,-0.866025,-0.500000,-0.207912,-0.978148,0.866025,-0.5,112,0.936881,-0.349647
1,2024-04-21 16:33:00,5.0,0.96,1004.54,34.22,51.05,27715.00,94.14,16,33,4,-0.866025,-0.500000,-0.309017,-0.951057,0.866025,-0.5,112,0.936881,-0.349647
2,2024-04-21 16:34:00,5.0,2.18,1004.49,34.04,51.10,29259.17,111.05,16,34,4,-0.866025,-0.500000,-0.406737,-0.913545,0.866025,-0.5,112,0.936881,-0.349647
3,2024-04-21 16:35:00,5.0,1.48,1004.53,34.20,51.05,34108.33,196.04,16,35,4,-0.866025,-0.500000,-0.500000,-0.866025,0.866025,-0.5,112,0.936881,-0.349647
4,2024-04-21 16:36:00,5.0,1.22,1004.56,34.16,51.18,29056.67,109.03,16,36,4,-0.866025,-0.500000,-0.587785,-0.809017,0.866025,-0.5,112,0.936881,-0.349647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1064448,10/22/2024 16:56,4.0,0.00,1007.65,27.67,100.00,387.50,0.09,16,56,10,-0.866025,-0.500000,-0.406737,0.913545,-0.866025,0.5,296,-0.927542,0.373720
1064449,10/22/2024 16:57,4.0,0.00,1007.70,27.65,100.00,355.00,0.08,16,57,10,-0.866025,-0.500000,-0.309017,0.951057,-0.866025,0.5,296,-0.927542,0.373720
1064450,10/22/2024 16:58,4.0,0.00,1007.71,27.61,100.00,329.17,0.08,16,58,10,-0.866025,-0.500000,-0.207912,0.978148,-0.866025,0.5,296,-0.927542,0.373720
1064451,10/22/2024 16:59,4.0,0.00,1007.71,27.58,100.00,306.67,0.07,16,59,10,-0.866025,-0.500000,-0.104528,0.994522,-0.866025,0.5,296,-0.927542,0.373720


In [11]:
import pandas as pd

# 假設 all_data 是你的 DataFrame
# 過濾出 LocationCode=2 的資料
location_2_data = all_data[all_data['LocationCode'] == 2]

# 取得 LocationCode=2 的所有時間戳
location_2_dates = location_2_data['DateTime']

# 過濾其他 LocationCode 的資料，只保留相同時間戳的資料
other_locations_data = all_data[all_data['LocationCode'] != 2]
other_locations_filtered = other_locations_data[other_locations_data['DateTime'].isin(location_2_dates)]

# 使用 merge 根據 DateTime 對齊資料，並重命名 Power(mW) 欄位
merged_data = pd.merge(location_2_data[['DateTime', 'Power(mW)']], 
                       other_locations_filtered[['DateTime', 'LocationCode', 'Power(mW)']],
                       on='DateTime', how='inner', suffixes=('_location_2', '_other_location'))

# 檢查合併後的資料，查看哪些 LocationCode 有重疊時間戳
print("合併後的資料:")
print(merged_data.head())

# 計算 LocationCode=2 的 Power 和其他 LocationCode 的 Power 之間的相關係數
correlations = {}
for loc_code, group in merged_data.groupby('LocationCode'):
    # 取得其他 LocationCode 的 Power(mW)，此時名稱已經被重命名
    other_power = group['Power(mW)_other_location']
    
    # 只考慮沒有缺失值的情況
    valid_data = group.dropna(subset=['Power(mW)_location_2', 'Power(mW)_other_location'])
    
    # 如果有效資料點不足，跳過這個 LocationCode
    if len(valid_data) < 2:
        print(f"LocationCode={loc_code} 有效資料點不足，無法計算相關係數")
        correlations[loc_code] = None
        continue

    # 計算相關係數
    location_2_power = valid_data['Power(mW)_location_2']
    other_power = valid_data['Power(mW)_other_location']
    
    correlation = location_2_power.corr(other_power)
    correlations[loc_code] = correlation

# 顯示相關係數
print("相關係數:")
print(correlations)


合併後的資料:
          DateTime  Power(mW)_location_2  LocationCode  \
0  1/17/2024 15:28             61.290000           8.0   
1  1/17/2024 15:29             62.200000           8.0   
2  1/17/2024 15:30             59.331429           8.0   
3  1/17/2024 15:31             56.462857           8.0   
4  1/17/2024 15:32             53.594286           8.0   

   Power(mW)_other_location  
0                     28.13  
1                     27.36  
2                     26.94  
3                     24.99  
4                     22.79  
相關係數:
{4.0: np.float64(0.8661555594164151), 7.0: np.float64(0.9583794994510939), 8.0: np.float64(0.6074486725839373), 9.0: np.float64(0.7214373930992135), 10.0: np.float64(0.8142153213061204), 12.0: np.float64(0.490881310231804)}


In [6]:
location_1_power

815289    0.07
815290    0.08
815291    0.08
815292    0.08
815293    0.07
          ... 
897992    0.02
897993    0.02
897994    0.02
897995    0.02
897996    0.03
Name: Power(mW), Length: 82708, dtype: float64